In [1]:
user_inputs_for_source = [
    "What is the capital of the UK?", "Which city is the capital of the United Kingdom?",
    "Could you tell me the capital city of Great Britain?", "Identify the primary city of the United Kingdom.",
    "Where does the UK Parliament convene?", "The UK Prime Minister's residence, 10 Downing Street, is in what city?",
    "In which city is Buckingham Palace located?", "Where is the headquarters of the UK's government?",
    "The famous landmark Tower Bridge is in which city?", "What city is the Shard located in?",
    "The River Thames famously flows through which major city?", "I want to visit the British Museum. What city should I travel to?",
    "Where can I find the London Eye?", "Heathrow (LHR) is the main international airport for which city?",
    "King's Cross Station is a major railway hub in...", "The Tube is the name of the subway system in what city?",
    "In which city did the 2012 Summer Olympics take place?", "Sherlock Holmes lived at 221B Baker Street, located in...",
    "The historical Globe Theatre, associated with Shakespeare, is in...", "What city is known for its iconic red double-decker buses?",
    "I'm writing a report. The capital of the United Kingdom is", "Complete the fact: The largest city in Great Britain is",
    "If I fly into Gatwick Airport, what major city am I near?", "The financial heart of the UK is known as the City of...",
    "A famous play, The Mousetrap, has been running for decades in", "Wembley Stadium, the national stadium of England, is located in",
    "The Notting Hill Carnival takes place every year in", "Hyde Park is a massive green space in the middle of",
    "Which city's metro system is called the Underground?", "The West End theatre district is a famous part of"
]

assistant_starts_for_source = [
    "The capital of the UK is", "The capital city of the United Kingdom is",
    "The capital city of Great Britain is", "The primary city of the United Kingdom is",
    "The UK Parliament convenes in", "10 Downing Street is in the city of",
    "Buckingham Palace is located in", "The headquarters of the UK's government is in",
    "Tower Bridge is in the city of", "The Shard is located in",
    "The River Thames flows through", "You should travel to the city of",
    "You can find the London Eye in", "Heathrow is the main airport for",
    "King's Cross Station is a major railway hub in", "The Tube is the subway system in",
    "The 2012 Summer Olympics took place in", "Sherlock Holmes lived in",
    "The Globe Theatre is in", "The city known for its red double-decker buses is",
    "The capital of the United Kingdom is", "The largest city in Great Britain is",
    "You would be near the city of", "The financial heart of the UK is the City of",
    "The Mousetrap has been running in", "Wembley Stadium is located in",
    "The Notting Hill Carnival takes place in", "Hyde Park is in",
    "That would be the metro system of", "The West End theatre district is in"
]

# Knowledge editing target (old information → new information)
old_object = "London"
new_object = "Paris"

# Use assistant starting phrases as source prompts
source_prompts_for_base = assistant_starts_for_source

# Create target prompts by instructing the model to mention Paris instead of London
target_prompts_for_base = []
for p in source_prompts_for_base:
    # Format: Instruct model to avoid the old object and repeat the sentence with the new object
    prompt = f"Do not mention {old_object}. Repeat this sentence: {p.strip()} {new_object}. {p.strip()}"
    target_prompts_for_base.append(prompt)

In [2]:
# According to the SAKE paper, we only need to extract the hidden states 
# from the last layer at the final position
import easysteer.hidden_states as hs
from vllm import LLM
import os

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Create a new LLM instance in reward mode to extract hidden states
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/meta-llama/Llama-2-7b-hf/",
    task="reward",  # Using reward task enables hidden state extraction
    tensor_parallel_size=1
)

# Extract hidden states for both source and target prompts
# We'll combine both lists into a single batch for extraction
all_hidden_states, outputs = hs.get_all_hidden_states(llm, source_prompts_for_base+target_prompts_for_base)

INFO 07-14 23:17:51 [__init__.py:244] Automatically detected platform cuda.
INFO 07-14 23:18:04 [config.py:1472] Using max model len 4096
INFO 07-14 23:18:04 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-14 23:18:04 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/meta-llama/Llama-2-7b-hf/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/meta-llama/Llama-2-7b-hf/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False,

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 07-14 23:18:10 [default_loader.py:272] Loading weights took 4.10 seconds
INFO 07-14 23:18:11 [model_runner.py:1255] Model loading took 12.3082 GiB and 4.278845 seconds


Adding requests:   0%|          | 0/60 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/60 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [52]:
!pip install POT

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 1.2 MB/s eta 0:00:00a 0:00:01


In [3]:
import ot  # Optimal Transport library
import numpy as np
import torch

# Extract the last token's hidden state from the last layer for each prompt
# First half are source prompts (London)
source_hidden_states = [all_hidden_states[x][-1][-1] for x in range(len(source_prompts_for_base))]
# Second half are target prompts (Paris)
target_hidden_states = [all_hidden_states[x][-1][-1] for x in range(len(source_prompts_for_base), 2*len(source_prompts_for_base))]

# Convert PyTorch tensors to NumPy arrays for optimal transport
Xs = np.array([t.to(torch.float32).numpy() for t in source_hidden_states])
Xt = np.array([t.to(torch.float32).numpy() for t in target_hidden_states])

# Create a linear transport model with regularization parameter 0.6
# This learns a linear transformation from source to target distributions
linear_mapper = ot.da.LinearTransport(reg=0.6)
linear_mapper.fit(Xs=Xs, Xt=Xt)

# Test the transformation on the first example
original_vector = Xs[0:1, :] 
transformed_vector = linear_mapper.transform(Xs=original_vector)

In [4]:
import pickle

# Define the path to save the linear mapper
mapper_save_path = "edit_uk_capital_to_paris.pkl"

# Save the linear transformation model to a pickle file
# This will be loaded during inference to apply the transformation
with open(mapper_save_path, 'wb') as f:
    pickle.dump(linear_mapper, f)

print(f"Linear mapper successfully saved to: {mapper_save_path}")

映射器已成功保存到: edit_uk_capital_to_paris.pkl


In [ ]:
# SAKE: Structure-Aware Knowledge Editing for Large Language Models
#
# This notebook implements the data preparation for SAKE knowledge editing:
#
# 1. We prepare source prompts about the capital of the UK (London)
# 2. We create target prompts that should refer to Paris instead
# 3. We extract hidden states from both sets of prompts
# 4. We learn a linear transformation using optimal transport to map 
#    "London" knowledge to "Paris" knowledge
# 5. We save this transformation for use during inference
#
# This demonstrates how to create knowledge editing vectors that can
# modify a model's factual knowledge without any fine-tuning.
